In [26]:
import cfscrape
from datetime import datetime
import csv

In [2]:
# Inflation
# https://sbcharts.investing.com/events_charts/us/1075.json

In [3]:
# GDP
# https://sbcharts.investing.com/events_charts/us/1076.json

In [4]:
scraper = cfscrape.create_scraper()
gdp_json = scraper.get('https://sbcharts.investing.com/events_charts/us/1076.json').json()
inflation_json = scraper.get('https://sbcharts.investing.com/events_charts/us/1075.json').json()

In [54]:
# clean gdp
gdp = []
for i in gdp_json['data']:
    dt_object = datetime.fromtimestamp(i[0]/1000)
    if dt_object.year >= 2014:
        temp = {
            'Date': dt_object.strftime('%Y-%m-%d'),
            'GDP': i[1]
        }
        gdp.append(temp)
gdp

[{'Date': '2014-01-30', 'GDP': 1.5},
 {'Date': '2014-05-29', 'GDP': 1.4},
 {'Date': '2014-08-28', 'GDP': 1.9},
 {'Date': '2014-11-27', 'GDP': 0.7},
 {'Date': '2015-01-29', 'GDP': 2.5},
 {'Date': '2015-05-28', 'GDP': 0.4},
 {'Date': '2015-08-27', 'GDP': 2},
 {'Date': '2015-11-26', 'GDP': 1.4},
 {'Date': '2016-01-28', 'GDP': 2.1},
 {'Date': '2016-05-19', 'GDP': 1.3},
 {'Date': '2016-08-18', 'GDP': 1.8},
 {'Date': '2016-11-17', 'GDP': 1.2},
 {'Date': '2017-01-26', 'GDP': 1.8},
 {'Date': '2017-05-18', 'GDP': 1.3},
 {'Date': '2017-08-17', 'GDP': 2},
 {'Date': '2017-11-16', 'GDP': 1.7},
 {'Date': '2018-01-23', 'GDP': 1.5},
 {'Date': '2018-05-10', 'GDP': 1.5},
 {'Date': '2018-08-09', 'GDP': 1.5},
 {'Date': '2018-11-08', 'GDP': 1.4},
 {'Date': '2019-01-24', 'GDP': 1.5},
 {'Date': '2019-05-09', 'GDP': 1},
 {'Date': '2019-08-08', 'GDP': 1.4},
 {'Date': '2019-11-07', 'GDP': 1.6},
 {'Date': '2020-01-23', 'GDP': 2.2},
 {'Date': '2020-05-07', 'GDP': -5.1},
 {'Date': '2020-08-06', 'GDP': -15.2},
 {'D

In [55]:
# clean inflation
inflation = []
for i in inflation_json['data']:
    dt_object = datetime.fromtimestamp(i[0]/1000)
    if dt_object.year >= 2014:
        temp = {
            'Date': dt_object.strftime('%Y-%m-%d'),
            'Inflation': i[1]
        }
        inflation.append(temp)
inflation

[{'Date': '2014-01-01', 'Inflation': 4.1},
 {'Date': '2014-02-01', 'Inflation': 4.2},
 {'Date': '2014-03-01', 'Inflation': 4.1},
 {'Date': '2014-04-04', 'Inflation': 3.9},
 {'Date': '2014-05-06', 'Inflation': 4.1},
 {'Date': '2014-06-05', 'Inflation': 4.5},
 {'Date': '2014-07-04', 'Inflation': 4.4},
 {'Date': '2014-08-05', 'Inflation': 4.9},
 {'Date': '2014-09-05', 'Inflation': 4.9},
 {'Date': '2014-10-08', 'Inflation': 4.4},
 {'Date': '2014-11-05', 'Inflation': 4.3},
 {'Date': '2014-12-05', 'Inflation': 3.7},
 {'Date': '2015-01-06', 'Inflation': 2.7},
 {'Date': '2015-02-05', 'Inflation': 2.4},
 {'Date': '2015-03-05', 'Inflation': 2.5},
 {'Date': '2015-04-07', 'Inflation': 2.4},
 {'Date': '2015-05-05', 'Inflation': 2.2},
 {'Date': '2015-06-05', 'Inflation': 1.6},
 {'Date': '2015-07-07', 'Inflation': 1.2},
 {'Date': '2015-08-05', 'Inflation': 0.8},
 {'Date': '2015-09-04', 'Inflation': 0.6},
 {'Date': '2015-10-06', 'Inflation': 0.4},
 {'Date': '2015-11-05', 'Inflation': 0.4},
 {'Date': '

In [56]:
import pandas as pd

In [57]:
gdp_df = pd.DataFrame(gdp)
gdp_df.set_index("Date", inplace=True)
gdp_df.to_csv('gdp.csv')

In [58]:
inflation_df = pd.DataFrame(inflation)
inflation_df.set_index("Date", inplace=True)
inflation_df.to_csv('inflation.csv')

In [103]:
inflation_df = pd.DataFrame(inflation)
inflation_df['Date'] = pd.to_datetime(inflation_df['Date'], format='%Y-%m-%d')
gdp_df = pd.DataFrame(gdp)
gdp_df['Date'] = pd.to_datetime(gdp_df['Date'], format='%Y-%m-%d')

res = pd.merge(inflation_df.assign(grouper=inflation_df['Date'].dt.to_period('M')),
                gdp_df.assign(grouper=gdp_df['Date'].dt.to_period('M')),
               on='grouper', how='left').drop('grouper', axis=1).drop('Date_y', axis=1)


res.rename(columns = {'Date_x':'Date'}, inplace = True)
res.set_index("Date", inplace=True)
res = res.interpolate(method='ffill')
res['GDP'] = res['GDP'].round(2)
res.to_csv('inflation_gdp.csv')
res

,Inflation,GDP
Date,,
2014-01-01,4.1,1.5
2014-02-01,4.2,1.5
2014-03-01,4.1,1.5
2014-04-04,3.9,1.5
2014-05-06,4.1,1.4
...,...,...
2023-01-05,8.1,2.0
2023-02-07,8.7,2.0
2023-03-07,8.6,2.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=104f152a-ece5-47f4-a4d4-d9ea9c0d19d6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>